# day 18

https://adventofcode.com/2019/day/18

In [1]:
import logging
import logging.config
import os

import yaml

In [2]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [3]:
FNAME = os.path.join('data', 'day18.txt')

LOGGER = logging.getLogger('day18')

## part 1

### problem statement:

#### loading data

In [381]:
test_0 = """#########
#b.A.@.a#
#########""", 8

test_1 = """########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################""", 86

test_2 = """########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################""", 132

test_3 = """#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################""", 136

test_4 = """########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################""", 81

tests = [test_0, test_1, test_2, test_3, test_4]

In [382]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read()

#### function def

In [383]:
import string

import networkx as nx

WALL = '#'
ORIGIN = '@'
EMPTY = '.'

KEYS = string.ascii_lowercase
DOORS = string.ascii_uppercase

INF = float('inf')

def map_to_graph(s):
    g = nx.Graph()
    for (i, row) in enumerate(s.split('\n')):
        for (j, c) in enumerate(row):
            if c != WALL:
                g.add_node((i, j), c=c)
    
    for (i, j) in g.nodes():
        neighbors = [(i - 1, j),
                     (i + 1, j),
                     (i, j - 1),
                     (i, j + 1)]
        for neighbor in neighbors:
            if neighbor in g:
                g.add_edge((i, j), neighbor)
    return g

In [384]:
def find_shit(g):
    key_nodes = {}
    door_nodes = {}

    for (n, d) in g.nodes(data=True):
        if d['c'] == ORIGIN:
            origin_node = n
        elif d['c'] in KEYS:
            key_nodes[d['c']] = n
        elif d['c'] in DOORS:
            door_nodes[d['c']] = n
    
    return origin_node, key_nodes, door_nodes

In [385]:
from collections import namedtuple
Rule = namedtuple('Rule', 'first,then')

In [386]:
def make_rules(g, origin_node, key_nodes, door_nodes):
    # for each key, find the doors that must be unlocked
    # to reach that key.
    rules = []
    for (key, key_node) in key_nodes.items():
        sp = nx.shortest_path(g, origin_node, key_node)
        for _ in sp:
            c = g.nodes[_]['c']
            if c in DOORS or (c in KEYS and c != key):
                rules.append(Rule(c.lower(), key))
    return rules

In [387]:
def subset_obeys_rules(subset, rules):
    idx = {c: i for (i, c) in enumerate(subset)}
    # rules will contain the majority of the keys in the map
    # the subset will not. we want our subset to obey all
    # rules that apply to them exclusively, meaning we are
    # only interested in rules where first and then are both
    # in our subset
    subset_rules = [r for r in rules
                    if r.first in subset
                    or r.then in subset]
    return all([idx.get(rule.first, INF) < idx.get(rule.then, INF)
                for rule in subset_rules])

In [388]:
g0 = map_to_graph(test_0[0])
origin_node_0, key_nodes_0, door_nodes_0 = find_shit(g0)
rules0 = make_rules(g0, origin_node_0, key_nodes_0, door_nodes_0)

In [389]:
assert subset_obeys_rules(('a', 'b'), rules0)
assert not subset_obeys_rules(('b', 'a'), rules0)

In [390]:
g1 = map_to_graph(test_1[0])
origin_node_1, key_nodes_1, door_nodes_1 = find_shit(g1)
rules1 = make_rules(g1, origin_node_1, key_nodes_1, door_nodes_1)

# one element
for k in 'abcdef':
    subset = (k,)
    assert subset_obeys_rules(subset, rules1) == (k == 'a')

# other examples
assert subset_obeys_rules(('a', 'b'), rules1)
assert not subset_obeys_rules(('b', 'a'), rules1)

assert subset_obeys_rules(('a', 'b', 'c'), rules1)
assert not subset_obeys_rules(('a', 'c', 'b'), rules1)
assert not subset_obeys_rules(('b', 'a', 'c'), rules1)

now we iteratively generate acceptable subsets (like shells), excluding any subsets that violate our rules

In [412]:
import tqdm.autonotebook as tqdm

def generate_valid_paths(keys, rules):
    keys = set(keys)
    shell = set([tuple()])
    for i in tqdm.trange(len(keys), leave=False):
        #LOGGER.debug(f'i = {i}')
        new_shell = set()
        for valid_subset_so_far in tqdm.tqdm(shell, leave=False):
            #LOGGER.debug(f'valid_subset_so_far = {valid_subset_so_far}')
            next_step_choices = keys.difference(valid_subset_so_far)
            for next_step_choice in next_step_choices:
                #LOGGER.debug(f'next_step_choice = {next_step_choice}')
                new_subset = valid_subset_so_far + (next_step_choice,)
                if subset_obeys_rules(new_subset, rules):
                    #LOGGER.debug('this works')
                    new_shell.add(new_subset)
        LOGGER.debug(f'found {len(new_shell)} valid new subsets of length {i + 1}')
        shell = new_shell
    return shell

In [413]:
LOGGER.setLevel(logging.DEBUG)
shell = generate_valid_paths(key_nodes_0, rules0)
LOGGER.setLevel(logging.INFO)
shell

2019-12-18 23:41:55,430 DEBUG    [day18.generate_valid_paths:18] found 1 valid new subsets of length 1


2019-12-18 23:41:55,476 DEBUG    [day18.generate_valid_paths:18] found 1 valid new subsets of length 2


{('a', 'b')}

In [414]:
print(test_1[0])

########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################


In [415]:
LOGGER.setLevel(logging.DEBUG)
shell = generate_valid_paths(key_nodes_1, rules1)
LOGGER.setLevel(logging.INFO)
shell

2019-12-18 23:41:57,972 DEBUG    [day18.generate_valid_paths:18] found 1 valid new subsets of length 1


2019-12-18 23:41:58,004 DEBUG    [day18.generate_valid_paths:18] found 1 valid new subsets of length 2


2019-12-18 23:41:58,049 DEBUG    [day18.generate_valid_paths:18] found 1 valid new subsets of length 3


2019-12-18 23:41:58,130 DEBUG    [day18.generate_valid_paths:18] found 2 valid new subsets of length 4


2019-12-18 23:41:58,173 DEBUG    [day18.generate_valid_paths:18] found 2 valid new subsets of length 5


2019-12-18 23:41:58,232 DEBUG    [day18.generate_valid_paths:18] found 2 valid new subsets of length 6


{('a', 'b', 'c', 'd', 'e', 'f'), ('a', 'b', 'c', 'e', 'd', 'f')}

In [416]:
def key_path_length(g, key_order, origin_node, key_nodes):
    n0 = origin_node
    pl = 0

    for key in key_order:
        n1 = key_nodes[key]
        p = nx.shortest_path_length(g, n0, n1)
        LOGGER.debug(f"dist from {g.nodes[n0]['c']} to {g.nodes[n1]['c']} is {p}")
        pl += p
        n0 = n1
    
    LOGGER.debug(f'key_order = {key_order}')
    LOGGER.debug(f'path lenght = {pl}')
    return pl

In [417]:
def q_1(data):
    g = map_to_graph(data)
    origin_node, key_nodes, door_nodes = find_shit(g)
    rules = make_rules(g, origin_node, key_nodes, door_nodes)
    
    best_path = None
    best_path_length = INF
    for valid_path in generate_valid_paths(key_nodes, rules):
        kpl = key_path_length(g, valid_path, origin_node, key_nodes)
        if kpl < best_path_length:
            best_path = valid_path
            best_path_length = kpl
    
    LOGGER.info(f'best key sequence: {best_path}')
    return best_path_length

#### tests

In [418]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    for test_map, ans in tests:
        LOGGER.info(f'\n{test_map}')
        assert q_1(test_map) == ans
    LOGGER.setLevel(logging.INFO)

In [419]:
test_q_1()

2019-12-18 23:42:04,162 INFO     [day18.test_q_1:4] 
#########
#b.A.@.a#
#########


2019-12-18 23:42:04,294 DEBUG    [day18.generate_valid_paths:18] found 1 valid new subsets of length 1


2019-12-18 23:42:04,325 DEBUG    [day18.generate_valid_paths:18] found 1 valid new subsets of length 2
2019-12-18 23:42:04,334 DEBUG    [day18.key_path_length:8] dist from @ to a is 2
2019-12-18 23:42:04,336 DEBUG    [day18.key_path_length:8] dist from a to b is 6
2019-12-18 23:42:04,336 DEBUG    [day18.key_path_length:12] key_order = ('a', 'b')
2019-12-18 23:42:04,339 DEBUG    [day18.key_path_length:13] path lenght = 8
2019-12-18 23:42:04,359 INFO     [day18.q_1:14] best key sequence: ('a', 'b')
2019-12-18 23:42:04,363 INFO     [day18.test_q_1:4] 
########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################


2019-12-18 23:42:04,433 DEBUG    [day18.generate_valid_paths:18] found 1 valid new subsets of length 1


2019-12-18 23:42:04,460 DEBUG    [day18.generate_valid_paths:18] found 1 valid new subsets of length 2


...

2019-12-18 23:42:14,796 DEBUG    [day18.generate_valid_paths:18] found 349392 valid new subsets of length 7


2019-12-18 23:42:52,405 DEBUG    [day18.generate_valid_paths:18] found 1822272 valid new subsets of length 8


2019-12-18 23:45:57,091 DEBUG    [day18.generate_valid_paths:18] found 9125760 valid new subsets of length 9


2019-12-18 23:59:20,275 DEBUG    [day18.generate_valid_paths:18] found 43355136 valid new subsets of length 10


KeyboardInterrupt: 

#### answer

loooool

brute force was
```
7582/15511210043330985984000000 [00:26<17115725575404552628:54:24, 251.74it/s]
```

In [421]:
q_1(load_data())

2019-12-19 01:07:17,839 DEBUG    [day18.generate_valid_paths:18] found 6 valid new subsets of length 1


2019-12-19 01:07:17,884 DEBUG    [day18.generate_valid_paths:18] found 33 valid new subsets of length 2


2019-12-19 01:07:17,982 DEBUG    [day18.generate_valid_paths:18] found 170 valid new subsets of length 3


2019-12-19 01:07:18,231 DEBUG    [day18.generate_valid_paths:18] found 843 valid new subsets of length 4


2019-12-19 01:07:19,007 DEBUG    [day18.generate_valid_paths:18] found 4081 valid new subsets of length 5


2019-12-19 01:07:22,674 DEBUG    [day18.generate_valid_paths:18] found 19107 valid new subsets of length 6


2019-12-19 01:07:40,597 DEBUG    [day18.generate_valid_paths:18] found 85176 valid new subsets of length 7


2019-12-19 01:09:08,086 DEBUG    [day18.generate_valid_paths:18] found 359490 valid new subsets of length 8


KeyboardInterrupt: 

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(data):
    return False

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == True
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin